In [ ]:
import deepchem as dc
import deepchem.models.layers as layers
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# TensorFlowでGPUを使用するための設定
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("GPU is available and configured.")
    except:
        print("GPU is available but configuration failed.")

# データフレームの作成（例）
data = {
    'smiles': ['CCO', 'CC', 'CCC', 'CCCC', 'CCCCC'],  # 例として5つのSMILES
    'objective': [0.1, 0.2, 0.15, 0.3, 0.25]  # それに対応する目標値
}
df = pd.DataFrame(data)

# データの分割
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# DeepChem用のデータセットの作成
featurizer = dc.feat.ConvMolFeaturizer()

# SMILES列と目標値を取得
train_smiles = train_df['smiles'].tolist()
train_targets = train_df['objective'].tolist()

test_smiles = test_df['smiles'].tolist()
test_targets = test_df['objective'].tolist()

# SMILESを特徴量に変換
train_features = featurizer.featurize(train_smiles)
test_features = featurizer.featurize(test_smiles)

# DeepChemのデータセット形式に変換
train_dataset = dc.data.NumpyDataset(train_features, np.array(train_targets))
test_dataset = dc.data.NumpyDataset(test_features, np.array(test_targets))

# カスタム層の設計
class CustomGraphConvModel(dc.models.KerasModel):
    def __init__(self, n_tasks, **kwargs):
        model = tf.keras.models.Sequential()
        model.add(layers.GraphConv(64, activation_fn=tf.nn.relu))
        model.add(layers.BatchNorm())
        model.add(layers.GraphPool())
        model.add(layers.GraphConv(64, activation_fn=tf.nn.relu))
        model.add(layers.BatchNorm())
        model.add(layers.GraphPool())
        model.add(layers.Dense(128, activation_fn=tf.nn.relu))
        model.add(layers.BatchNorm())
        model.add(layers.Dense(n_tasks, activation_fn=None))
        
        loss = dc.models.losses.L2Loss()
        super(CustomGraphConvModel, self).__init__(model, loss, **kwargs)

# モデルの作成
model = CustomGraphConvModel(n_tasks=1, mode='regression')

# モデルのトレーニング
model.fit(train_dataset, nb_epoch=100)

# 予測
train_pred = model.predict(train_dataset)
test_pred = model.predict(test_dataset)

# 評価
metric = dc.metrics.Metric(dc.metrics.mae_score)
train_score = model.evaluate(train_dataset, [metric])
test_score = model.evaluate(test_dataset, [metric])

print("Train MAE: ", train_score)
print("Test MAE: ", test_score)

# 予実プロットの作成
def plot_predictions(y_true, y_pred, title="Predicted vs Actual"):
    plt.figure(figsize=(8, 6))
    plt.scatter(y_true, y_pred, alpha=0.7)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], '--', color='red')
    plt.xlabel("Actual Values")
    plt.ylabel("Predicted Values")
    plt.title(title)
    plt.grid(True)
    plt.show()

# トレーニングデータの予実プロット
plot_predictions(train_targets, train_pred, title="Training Data: Predicted vs Actual")

# テストデータの予実プロット
plot_predictions(test_targets, test_pred, title="Test Data: Predicted vs Actual")


In [ ]:
import deepchem as dc
import deepchem.models.layers as layers
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# データフレームの作成（例）
data = {
    'smiles': ['CCO', 'CC', 'CCC', 'CCCC', 'CCCCC'],  # 例として5つのSMILES
    'objective': [0.1, 0.2, 0.15, 0.3, 0.25]  # それに対応する目標値
}
df = pd.DataFrame(data)

# データの分割
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# DeepChem用のデータセットの作成
featurizer = dc.feat.ConvMolFeaturizer()

# SMILES列と目標値を取得
train_smiles = train_df['smiles'].tolist()
train_targets = train_df['objective'].tolist()

test_smiles = test_df['smiles'].tolist()
test_targets = test_df['objective'].tolist()

# SMILESを特徴量に変換
train_features = featurizer.featurize(train_smiles)
test_features = featurizer.featurize(test_smiles)

# DeepChemのデータセット形式に変換
train_dataset = dc.data.NumpyDataset(train_features, np.array(train_targets))
test_dataset = dc.data.NumpyDataset(test_features, np.array(test_targets))

# カスタム層の設計
class CustomGraphConvModel(dc.models.KerasModel):
    def __init__(self, n_tasks, **kwargs):
        model = tf.keras.models.Sequential()
        model.add(layers.GraphConv(64, activation_fn=tf.nn.relu))
        model.add(layers.BatchNorm())
        model.add(layers.GraphPool())
        model.add(layers.GraphConv(64, activation_fn=tf.nn.relu))
        model.add(layers.BatchNorm())
        model.add(layers.GraphPool())
        model.add(layers.Dense(128, activation_fn=tf.nn.relu))
        model.add(layers.BatchNorm())
        model.add(layers.Dense(n_tasks, activation_fn=None))
        
        loss = dc.models.losses.L2Loss()
        super(CustomGraphConvModel, self).__init__(model, loss, **kwargs)

# モデルの作成
model = CustomGraphConvModel(n_tasks=1, mode='regression')

# モデルのトレーニング
model.fit(train_dataset, nb_epoch=100)

# 予測
train_pred = model.predict(train_dataset)
test_pred = model.predict(test_dataset)

# 評価
metric = dc.metrics.Metric(dc.metrics.mae_score)
train_score = model.evaluate(train_dataset, [metric])
test_score = model.evaluate(test_dataset, [metric])

print("Train MAE: ", train_score)
print("Test MAE: ", test_score)

# 予実プロットの作成
def plot_predictions(y_true, y_pred, title="Predicted vs Actual"):
    plt.figure(figsize=(8, 6))
    plt.scatter(y_true, y_pred, alpha=0.7)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], '--', color='red')
    plt.xlabel("Actual Values")
    plt.ylabel("Predicted Values")
    plt.title(title)
    plt.grid(True)
    plt.show()

# トレーニングデータの予実プロット
plot_predictions(train_targets, train_pred, title="Training Data: Predicted vs Actual")

# テストデータの予実プロット
plot_predictions(test_targets, test_pred, title="Test Data: Predicted vs Actual")
